# Introduction
Welcome to the SQL mini project. For this project, you will use
Springboard' online SQL platform, which you can log into through the
following link:

https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

Note that, if you need to, you can also download these tables locally.

In the mini project, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables.

In [1]:
import pandas as pd
import sqlite3

In [2]:
bookings = pd.read_csv('country_club.csv', names=['bookid', 'facid', 'memid', 'starttime', 'slots'], nrows=4043)
facilities = pd.read_csv('country_club.csv', names=['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance'], skiprows=4043, nrows=9)
members = pd.read_csv('country_club.csv', names=['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate'], skiprows=4052, nrows=31)

In [3]:
db = 'country_club.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

In [4]:
def show_tables():
    q = '''
    SELECT name, type
    FROM sqlite_master
    WHERE type IN ("table", "view")
    '''
    return run_query(q)
show_tables()

,name,type
0,bookings,table
1,facilities,table
2,members,table


In [5]:
tables = {
    'bookings': bookings,
    'facilities': facilities,
    'members': members }

with sqlite3.connect(db) as conn:
    for name, data in tables.items():
        conn.execute('DROP TABLE IF EXISTS {};'.format(name))
        data.to_sql(name, conn, index=False)
show_tables()

,name,type
0,bookings,table
1,facilities,table
2,members,table


### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [7]:
q1 = '''
        SELECT * FROM facilities
        WHERE membercost != 0
'''
run_query(q1)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


### Q2: How many facilities do not charge a fee to members?

In [27]:
q2 = '''
        SELECT COUNT(*) FROM facilities
        WHERE membercost = 0
'''
run_query(q2)

,COUNT(*)
0,4


### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [29]:
q3 = '''
        SELECT 
            facid, 
            name, 
            membercost, 
            monthlymaintenance 
        FROM facilities
        WHERE membercost < .2*monthlymaintenance
'''
run_query(q3)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [78]:
q4 = '''
        SELECT * FROM facilities
        WHERE name LIKE '%2%'
'''
run_query(q4)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [49]:
q5 = '''
        SELECT name, monthlymaintenance,
        CASE
            WHEN monthlymaintenance < 100 THEN 'cheap'
            ELSE 'expensive'
            END
            AS price_category
            FROM facilities
'''
run_query(q5)

,name,monthlymaintenance,price_category
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [50]:
q6 = '''
        SELECT m.firstname || ' ' || m.surname customer_name 
        FROM members m
        ORDER BY joindate DESC
'''
run_query(q6)

,customer_name
0,Darren Smith
1,Erica Crumpet
2,John Hunt
3,Hyacinth Tupperware
4,Millicent Purview
5,Henry Worthington-Smyth
6,David Farrell
7,Henrietta Rumney
8,Douglas Jones
9,Ramnaresh Sarwin


### Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [10]:
q7 = '''
        WITH facilities_booked AS
        (
        SELECT b.facid, b.memid, 
            m.firstname || ' ' || m.surname member_name,
            f.name facility
        FROM bookings b
        INNER JOIN members m ON m.memid = b.memid
        INNER JOIN facilities f ON f.facid = b.facid
        )
        
        SELECT DISTINCT member_name, facility
        FROM facilities_booked
        WHERE facility = 'Tennis Court 1' OR facility = 'Tennis Court 2'
        ORDER BY member_name
        
        
'''
run_query(q7)

,member_name,facility
0,Anne Baker,Tennis Court 1
1,Anne Baker,Tennis Court 2
2,Burton Tracy,Tennis Court 1
3,Burton Tracy,Tennis Court 2
4,Charles Owen,Tennis Court 1
5,Charles Owen,Tennis Court 2
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 1
8,David Farrell,Tennis Court 2
9,David Jones,Tennis Court 1


### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [98]:
q8 = '''
        /* first let's create a table with the total price of each booking, then we can use a WHERE clause with a
        LIKE statement to filter out total price of bookings greater than $30 for the date of interest */
            
        WITH booked_prices AS
        (
        SELECT 
            b.facid, 
            b.memid, 
            b.slots,
            b.starttime,
            m.firstname || ' ' || m.surname member_name,
            f.name facility,
            f.membercost,
            f.guestcost,
            CASE
                WHEN b.memid = 0 THEN b.slots*guestcost
                ELSE b.slots*membercost
            END
            AS total_cost
        FROM bookings b
        INNER JOIN members m ON m.memid = b.memid
        INNER JOIN facilities f ON f.facid = b.facid
        )
        
        SELECT
            facility,
            member_name,
            total_cost
        FROM booked_prices
        WHERE total_cost > 30 AND starttime LIKE '%2012-09-14%'
        
'''
run_query(q8)

,facility,member_name,total_cost
0,Tennis Court 1,GUEST GUEST,75.0
1,Tennis Court 1,GUEST GUEST,75.0
2,Tennis Court 2,GUEST GUEST,75.0
3,Tennis Court 2,GUEST GUEST,150.0
4,Massage Room 1,GUEST GUEST,160.0
5,Massage Room 1,GUEST GUEST,160.0
6,Massage Room 1,Jemima Farrell,39.6
7,Massage Room 1,GUEST GUEST,160.0
8,Massage Room 2,GUEST GUEST,320.0
9,Squash Court,GUEST GUEST,70.0


### Q9: This time, produce the same result as in Q8, but using a subquery.

In [83]:
q9 = '''
        SELECT
            facility,
            member_name,
            total_cost
        FROM 
        (
        SELECT 
            b.facid, 
            b.memid, 
            b.slots,
            b.starttime,
            m.firstname || ' ' || m.surname member_name,
            f.name facility,
            f.membercost,
            f.guestcost,
            CASE
                WHEN b.memid = 0 THEN b.slots*guestcost
                ELSE b.slots*membercost
            END
            AS total_cost
        FROM bookings b
        INNER JOIN members m ON m.memid = b.memid
        INNER JOIN facilities f ON f.facid = b.facid
        )
        WHERE total_cost > 30 AND starttime LIKE '%2012-09-14%'
        
'''
run_query(q9)

,facility,member_name,total_cost
0,Tennis Court 1,GUEST GUEST,75.0
1,Tennis Court 1,GUEST GUEST,75.0
2,Tennis Court 2,GUEST GUEST,75.0
3,Tennis Court 2,GUEST GUEST,150.0
4,Massage Room 1,GUEST GUEST,160.0
5,Massage Room 1,GUEST GUEST,160.0
6,Massage Room 1,Jemima Farrell,39.6
7,Massage Room 1,GUEST GUEST,160.0
8,Massage Room 2,GUEST GUEST,320.0
9,Squash Court,GUEST GUEST,70.0


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [17]:
q10 = '''
        /* first let's create a table with the total price of each booking, and then we can sum over the price of
            all the bookings and group by facility */
    
        WITH booked_prices AS
        (
        SELECT 
            b.facid, 
            b.memid, 
            b.slots,
            f.name facility,
            f.membercost,
            f.guestcost,
            CASE
                WHEN b.memid = 0 THEN b.slots*guestcost
                ELSE b.slots*membercost
            END
            AS total_cost
        FROM bookings b
        LEFT JOIN facilities f ON f.facid = b.facid
        ),
        
        revenue_per_facility AS
        (
        SELECT
            facility, SUM(total_cost) total_revenue
        FROM booked_prices
        GROUP BY facility
        ORDER BY total_revenue DESC
        )
        
        SELECT *
        FROM revenue_per_facility
        WHERE total_revenue < 1000
'''
run_query(q10)

,facility,total_revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0
